In [1]:
import hashlib
import random

# Define the Schnorr signature protocol
def schnorr_signature(msg, priv_key):
    r = random.randint(1, q - 1)  # Choose a random nonce r
    R = pow(g, r, p)  # Compute R = g^r mod p
    h = int(hashlib.sha256(msg.encode()).hexdigest(), 16)  # Compute the hash of the message
    s = (r + priv_key * h) % q  # Compute s = r + x * h mod q
    return (R, s)

# Define the NIZKP protocol
def nizkp(msg, R, s):
    c = int(hashlib.sha256((str(g) + str(R) + msg).encode()).hexdigest(), 16)  # Compute the challenge c
    z = (s + c * priv_key) % q  # Compute the response z = s + x * c mod q
    return (c, z)

# Define the verification function for the NIZKP protocol
def verify_nizkp(msg, R, c, z):
    # Recompute the challenge c
    c_prime = int(hashlib.sha256((str(g) + str(R) + msg).encode()).hexdigest(), 16)
    # Verify that the challenge is the same as the original
    if c_prime != c:
        return False
    # Verify that z = s + x * c mod q
    V = (pow(g, z, p) * pow(y, c, p)) % p
    if V == R:
        return True
    else:
        return False

In [2]:
!pip install Crypto

from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric.utils import (
    encode_dss_signature, decode_dss_signature)
from cryptography.hazmat.primitives.serialization import (
    Encoding, PublicFormat)
from cryptography.hazmat.backends import default_backend
from cryptography.exceptions import InvalidSignature

import secrets

# Generate a random 256-bit prime number
def generate_prime():
    while True:
        p = secrets.randbits(256)
        if is_prime(p):
            return p

# Check if a number is prime
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5)+1):
        if n % i == 0:
            return False
    return True

# Generate a random generator for a prime field
def generate_generator(p):
    g = 2
    while pow(g, (p-1)//2, p) == 1 or pow(g, 2, p) == 1:
        g += 1
    return g

# Generate a private/public key pair
def generate_key_pair(p, g):
    x = secrets.randbelow(p-1)
    y = pow(g, x, p)
    return x, y

# Compute the challenge value for the proof
def compute_challenge(*args):
    h = hashes.Hash(hashes.SHA256(), backend=default_backend())
    for arg in args:
        h.update(int.to_bytes(arg, 32, byteorder='big'))
    return int.from_bytes(h.finalize(), byteorder='big')

# Compute the proof of knowledge for a discrete logarithm
def prove_dlog(p, g, x):
    # Generate a random number r
    r = secrets.randbelow(p-1)
    # Compute the commitment value
    A = pow(g, r, p)
    # Compute the challenge value
    c = compute_challenge(p, g, A)
    # Compute the response value
    z = (r + c*x) % (p-1)
    # Return the proof (A, z)
    return A, z

# Verify the proof of knowledge for a discrete logarithm
def verify_dlog(p, g, y, proof):
    A, z = proof
    # Check that A is a valid element in the prime field
    if A <= 0 or A >= p:
        return False
    # Compute the challenge value
    c = compute_challenge(p, g, A)
    # Check that A is a valid commitment value
    if pow(g, z, p) != (A * pow(y, c, p)) % p:
        return False
    # The proof is valid
    return True


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 KB 9.8 MB/s eta 0:00:00


In [3]:
import numpy as np
import sympy as sp

# Define the variables
p = 17
q = 19
g = 3

# Define the functions
def mod_inverse(a, m):
    return pow(a, -1, m)

def mod_exp(base, exponent, modulus):
    return pow(base, exponent, modulus)

# Define the NIZKP module
def nizkp_module(a, b, r):
    A = mod_exp(g, a, p)
    B = mod_exp(g, b, p)
    c = np.random.randint(0, q)
    R = mod_exp(g, r, p)
    T1 = mod_exp(A, c, p)
    T2 = mod_exp(B, c, p)
    S = (R * mod_exp(T1, b, p) * mod_exp(T2, mod_inverse(r, q), p)) % p
    return A, B, R, S, c

# Test the NIZKP module
a = np.random.randint(0, q)
b = np.random.randint(0, q)
r = np.random.randint(0, q)
print(nizkp_module(a, b, r))

(4, 2, 9, 8, 6)
